# SAL Script - Slew and Track

Performs a single slew and track using SAL Script.  
It can be used as a minimum functionality prototype for the Soak Test.

## Notebook Setup

Start by importing relevat libraries, setting up domain, remotes, logs, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging
import yaml

import numpy as np

from astropy import units as u
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.standardscripts.maintel.track_target import TrackTarget

In [ ]:
track_target = TrackTarget(index=-20221117)
await track_target.start_task

In [ ]:
mtcs = track_target.tcs

## Prepare Components

In [ ]:
await mtcs.set_state(salobj.State.DISABLED, components=["mtmount", "mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtmount", "mtrotator"])

In [ ]:
az = mtcs.rem.mtmount.tel_azimuth.get().actualPosition
el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

print(az, el)

In [ ]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=2, elevation=62)

In [ ]:
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)

## Helper Functions

In [ ]:
def random_walk_azel_by_time(total_time, 
                             radius=3.5, 
                             min_az=-200., 
                             max_az=+200, 
                             min_el=30, 
                             max_el=80):
    """
    Generate Az/El coordinates for a a long time so we can slew and track 
    to these targets. 
    
    Parameters
    ----------
    total_time : `float`
        Total observation time in seconds.
    radius : `float`, default 3.5
        Radius of the circle where the next coordinate will fall.
    min_az :  `float`, default -200 
        Minimum accepted azimuth in deg.
    max_az :  `float`, default +200
        Maximum accepted azimuth in deg.
    min_el : `float`, default 30
        Minimum accepted elevation in deg.
    max_el : `float`, default 80
        Maximum accepted elevation in deb.
    """

    track_target.log.info(
        f"{'Time':25s}{'Steps':>10s}{'New Az':>10s}{'New El':>10s}")
        
    step = 0
    timer_task = asyncio.create_task(asyncio.sleep(total_time))
    
    while not timer_task.done():
        
        current_az = mtcs.rem.mtmount.tel_azimuth.get()
        current_az = current_az.actualPosition
        # current_az = _az
        offset_az = np.sqrt(radius) * (2 * np.random.rand() - 1)
        new_az = current_az + offset_az
                
        current_el = mtcs.rem.mtmount.tel_elevation.get()
        current_el = current_el.actualPosition
        # current_el = _el
        offset_el = np.sqrt(radius) * (2 * np.random.rand() - 1)
        new_el = current_el + offset_el
        
        if new_az <= min_az or new_az >= max_az:
            new_az = current_az - offset_az
            
        if new_el <= min_el or new_el >= max_el:
            new_el = current_el - offset_el

        t = Time.now().to_value("isot")
        track_target.log.info(
            f"{t:25s}{step:10d}{new_az:10.2f}{new_el:10.2f}")

        yield new_az, new_el
        step += 1

In [ ]:
async def slew_and_track(az, el, track_for):
    """Use the `TrackTarget` standard script to slew and track 
    one coordinate emulating the Script Queue.
    
    Parameters
    ----------
    az : `float`
        Azimuth in hour angle.
    el :  `float`
        Elevation in degrees.
    track_for : `float` 
        Number of seconds to track for.
    """
    radec = track_target.tcs.radec_from_azel(az, el)
    
    configuration = yaml.safe_dump(
        {
            "slew_icrs": {
                "ra": float(radec.ra.hour),
                "dec": float(radec.dec.deg),
            }, 
            "rot_value": 0., 
            "rot_type": "Physical", 
            "track_for": track_for, 
            "stop_when_done": True,
            # "ignore": [
            #     "mtaos",
            #     "mtdome",
            #     "mtdometrajectory",
            #     "mthexapod_1",
            #     "mthexapod_2", 
            #     "mtm1m3", 
            #     "mtm2",
            # ],
        }
    )

    # Set script state to UNCONFIGURED
    # this is required to run the script a 2nd time but otherwise is a no-op
    await track_target.set_state(ScriptState.UNCONFIGURED)

    # Configure the script, which puts the ScriptState to CONFIGURED
    config_data = track_target.cmd_configure.DataType()
    config_data.config = configuration

    await track_target.do_configure(config_data)
    results = await track_target.run()

## Run Observation Simulation

In [ ]:
for az, el in random_walk_azel_by_time(120.):
    # await asyncio.sleep(10)
    await slew_and_track(az, el, 30)